In [14]:
from torch.autograd import Variable
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim

In [15]:
context_size = 3
embed_size = 2
xmax = 2
alpha = 0.75
batch_size = 20
l_rate = 0.001
num_epochs = 2

In [16]:
corpus = ["Golovkin won his first major world championship, the WBA interim middleweight title, by defeating Milton Núñez in 2010. The WBA elevated him to Regular champion status in the same year. In 2014, Golovkin was elevated to the status of WBA (Super) champion and successfully defended both his titles against Daniel Geale. Later that year he defeated Marco Antonio Rubio to win WBC interim middleweight title, and defeated David Lemieux for the IBF middleweight title in 2015. After Canelo Álvarez vacated his WBC middleweight title in 2016, Golovkin was elevated to full champion and held three of the four major world titles in boxing until being stripped by the IBF in 2018 for not fighting Serhiy Derevianchenko. Golovkin lost all his belts in 2018 following a loss to Alvarez but regained his IBF and IBO titles by defeating Derevianchenko in 2019."]

In [17]:
#corpus делится на tokens(слова) через точки и пробелы
tokens = []
for i in range(len(corpus)):
    sents = corpus[i].split(".")
    for j in range(len(sents)):
        tokens = tokens +sents[j].lower().split(" ")

In [18]:
#команда создает словарь
my_dict = list(set(tokens))

In [19]:
#присвоение индекса к словам
word2idx={}
idx2word={}
for ind,token in enumerate(my_dict):
    word2idx[token]=ind
    idx2word[ind]=token

In [20]:
#создается нулевая таблица
ss=len(my_dict)
zero_matrix=np.zeros((ss,ss))
#заполняется таблица совстречаемости 
for idx in range (len(tokens)-1):
    ind1 = word2idx[tokens[idx]]
    ind2 = word2idx[tokens[idx+1]]
    zero_matrix[ind1,ind2]+=1

In [21]:
#транспонирование таблицы
coocs = np.transpose(np.nonzero(zero_matrix))

In [22]:
#вычисление весов
def wf(x):
    if x < xmax:
        return (x/xmax)**alpha
    return 1

In [23]:
#вычисление левых и правых слов
vocab_size=len(my_dict)
w_list_size=len(tokens)
left_emb, right_emb = [
    [Variable(torch.from_numpy(np.random.normal(0, 0.01, (embed_size, 1))),
        requires_grad = True) for j in range(vocab_size)] for i in range(2)]
left_biases, right_biases = [
    [Variable(torch.from_numpy(np.random.normal(0, 0.01, 1)), 
        requires_grad = True) for j in range(vocab_size)] for i in range(2)]

In [24]:
optimize = optim.Adam(left_emb + right_emb + left_biases + right_biases, lr = l_rate)

In [25]:
def gen_batch():
    sample = np.random.choice(np.arange(len(coocs)), size=batch_size, replace=False)
    left_vector, right_vector, cov, left_vector_biases, right_vector_biases = [], [], [], [], []
    for chosen in sample:
        ind = tuple(coocs[chosen])
        left_vector.append(left_emb[ind[0]])
        right_vector.append(right_emb[ind[1]])
        cov.append(zero_matrix[ind])
        left_vector_biases.append(left_biases[ind[0]])
        right_vector_biases.append(right_biases[ind[1]])
    return left_vector, right_vector, cov, left_vector_biases, right_vector_biases

In [26]:
# тренировочная модель
for epoch in range(num_epochs):
    num_batches = int(w_list_size/batch_size)
    average_loss = 0.0
    for batch in range(num_batches):
        optimize.zero_grad()
        left_vector, right_vector, cov, left_vector_biases, right_vector_biases = gen_batch()
        loss = sum([torch.mul((torch.dot(left_vector[i].view(-1), right_vector[i].view(-1)) +
                left_vector_biases[i] + right_vector_biases[i] - np.log(cov[i]))**2,
                wf(cov[i])) for i in range(batch_size)])
        average_loss += loss.data[0]/num_batches
        loss.backward()
        optimize.step()

In [27]:
# визуализация
if embed_size == 2:
    word_inds = np.random.choice(np.arange(len(my_dict)), size=10, replace=False)
    for word_ind in word_inds:
        w_embed = (left_emb[word_ind].data + right_emb[word_ind].data).numpy()
        x, y = w_embed[0][0], w_embed[1][0]
        plt.scatter(x, y)
        plt.annotate(my_dict[word_ind], xy=(x, y), xytext=(5, 2),
            textcoords='offset points', ha='right', va='bottom')
    plt.savefig("test_pic.png")